In [ ]:
!gdown --id 1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
!mv _sqlite3.cpython-37m-x86_64-linux-gnu.so /usr/lib/python3.7/lib-dynload/
import os
os.kill(os.getpid(), 9)

В июле проводилась специальная акция, направленная на улучшение показателя Retention. Новым пользователям присылали письмо с купоном. Какой эффект от данной акции?

# Импорт библиотек

In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [2]:
con = sqlite3.connect('db')

Объект курсора используется для выполнения команд и запросов SQL

In [3]:
cur = con.cursor()

Функция SELECT

In [4]:
def select(sql):
  return pd.read_sql(sql,con)

Загрузка данных

https://drive.google.com/file/d/1efmNDUGsMpyHTTXK2PXl_32Exk4IdmRX/view?usp=sharing

In [5]:
! gdown --id 1efmNDUGsMpyHTTXK2PXl_32Exk4IdmRX

Downloading...
From: https://drive.google.com/uc?id=1efmNDUGsMpyHTTXK2PXl_32Exk4IdmRX
To: /content/analyst_test_transactions_db.csv
100% 3.07M/3.07M [00:00<00:00, 98.8MB/s]


In [6]:
df = pd.read_csv('/content/analyst_test_transactions_db.csv')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76857 entries, 0 to 76856
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           76857 non-null  int64  
 1   order_id          76857 non-null  float64
 2   reg_date          76857 non-null  object 
 3   transaction_date  76857 non-null  object 
 4   revenue           76857 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 2.9+ MB


Перевод в формат ДАТЫ

In [9]:
df['reg_date'] = pd.to_datetime(df['reg_date'],format='%Y-%m-%d')
df['transaction_date'] = pd.to_datetime(df['transaction_date'],format='%Y-%m-%d')

In [12]:
df.to_sql('trans',con,index=False,if_exists='replace')

In [13]:
sql = '''
SELECT
  * 
FROM 
  trans t'''

In [14]:
select(sql)

,user_id,order_id,reg_date,transaction_date,revenue
0,57562,94545.0,2013-12-22 00:00:00,2015-07-16 00:00:00,1088
1,40047,81824.0,2013-11-24 00:00:00,2015-01-19 00:00:00,1027
2,48153,144851.0,2015-07-03 00:00:00,2015-10-29 00:00:00,601
3,54591,116111.0,2013-07-12 00:00:00,2015-04-30 00:00:00,881
4,39789,134943.0,2013-05-25 00:00:00,2015-07-21 00:00:00,984
...,...,...,...,...,...
76852,36921,163682.0,2015-01-15 00:00:00,2015-03-28 00:00:00,1095
76853,31360,145146.0,2015-01-04 00:00:00,2015-08-09 00:00:00,547
76854,56195,8929.0,2012-09-18 00:00:00,2015-11-12 00:00:00,850
76855,46362,78673.0,2015-02-20 00:00:00,2015-07-21 00:00:00,356


In [15]:
sql = '''
SELECT 
  t.user_id, 
  DATE(MIN(reg_date),'start of month') AS reg_month 
FROM 
  trans t
GROUP BY 
  1'''

In [16]:
select(sql)

,user_id,reg_month
0,-37599,2015-01-01
1,-32239,2015-11-01
2,-29822,2015-07-01
3,-29531,2014-09-01
4,-25337,2015-04-01
...,...,...
48209,128964,2015-10-01
48210,130617,2015-02-01
48211,131071,2012-04-01
48212,132484,2015-06-01


In [17]:
min = '''
SELECT 
  DATE(MIN(t.transaction_date),'start of month') 
FROM 
  trans t'''

In [18]:
max = '''
SELECT 
  DATE(MAX(t.transaction_date),'start of month') 
FROM 
  trans t'''

In [20]:
select(min)

,"DATE(MIN(t.transaction_date),'start of month')"
0,2015-01-01


In [19]:
select(max)

,"DATE(MAX(t.transaction_date),'start of month')"
0,2015-11-01


Генерация диапазона дат (календарь)

In [23]:
sql = f'''
WITH RECURSIVE dates(date) AS (
  VALUES(({min}))
  
  UNION ALL

  SELECT 
    date(date, '+1 month')
  FROM 
    dates
  WHERE 
    date < ({max})
)
SELECT 
  date 
FROM 
  dates'''

In [24]:
select(sql)

,date
0,2015-01-01
1,2015-02-01
2,2015-03-01
3,2015-04-01
4,2015-05-01
5,2015-06-01
6,2015-07-01
7,2015-08-01
8,2015-09-01
9,2015-10-01


In [32]:
sql = f'''
WITH users AS (
  SELECT 
    t.user_id, 
    DATE(MIN(reg_date),'start of month') AS reg_month 
  FROM 
    trans t
  GROUP BY 
    1),

dates AS (
WITH RECURSIVE dates(date) AS (
  VALUES(({min}))
  UNION ALL
  SELECT 
    DATE(date, '+1 month')
  FROM 
    dates
  WHERE 
    date < ({max})
)
  SELECT 
    date 
  FROM 
    dates

),

template AS (
  SELECT 
    t.user_id, 
    t.reg_month, 
    d.date AS month 
  FROM 
    users t
  JOIN 
    dates d ON d.date >= t.reg_month),

trans_month AS (
  SELECT 
    t.user_id,
    DATE(t.transaction_date,'start of month') AS month,
    SUM(t.revenue) AS revenue_sum,
    COUNT(*) AS transaction_cnt
  FROM 
    trans t
  GROUP BY
    1, 2
),

report AS (
  SELECT 
    t.*, 
    tm.revenue_sum, 
    tm.transaction_cnt,
    CASE WHEN tm.revenue_sum > 0 THEN 1 ELSE 0 END AS active,
    ROUND((JULIANDAY(t.month) - JULIANDAY(t.reg_month)) / 30) AS lifetime
  FROM 
    template t
  LEFT JOIN 
    trans_month tm ON t.user_id = tm.user_id AND t.month = tm.month)

SELECT 
  t.reg_month, 
  t.lifetime, 
  AVG(t.active) AS retention 
FROM 
  report t
WHERE 
  t.reg_month >= '2015-01-01'
  --AND t.user_id = 128006
GROUP BY 
  1,2
'''

In [33]:
select(sql)

,reg_month,lifetime,retention
0,2015-01-01,0.0,0.116928
1,2015-01-01,1.0,0.116056
2,2015-01-01,2.0,0.129581
3,2015-01-01,3.0,0.116492
4,2015-01-01,4.0,0.115183
...,...,...,...
61,2015-09-01,1.0,0.381497
62,2015-09-01,2.0,0.221414
63,2015-10-01,0.0,0.561974
64,2015-10-01,1.0,0.448857


In [34]:
t = select(sql)

In [35]:
t.pivot_table(index='reg_month',columns='lifetime',values='retention',aggfunc='max')

lifetime,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0
reg_month,,,,,,,,,,,
2015-01-01,0.116928,0.116056,0.129581,0.116492,0.115183,0.124782,0.118237,0.131326,0.126091,0.148778,0.114311
2015-02-01,0.132856,0.141543,0.135922,0.121615,0.122126,0.119571,0.123659,0.149208,0.163005,0.105263,NaN
2015-03-01,0.164114,0.138403,0.129650,0.127462,0.138403,0.145514,0.138950,0.170678,0.117068,NaN,NaN
2015-04-01,0.193627,0.158088,0.144608,0.136029,0.161152,0.137255,0.167892,0.127451,NaN,NaN,NaN
2015-05-01,0.246457,0.184252,0.154331,0.136220,0.152756,0.159055,0.132283,NaN,NaN,NaN,NaN
2015-06-01,0.257924,0.216283,0.180236,0.168428,0.189559,0.141703,NaN,NaN,NaN,NaN,NaN
2015-07-01,0.283110,0.331367,0.179088,0.181769,0.124397,NaN,NaN,NaN,NaN,NaN,NaN
2015-08-01,0.423358,0.268613,0.222628,0.157664,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-09-01,0.429314,0.381497,0.221414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
